In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

# !pip install -q -U keras-tuner
import kerastuner as kt

In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

    return model

#### Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - RandomSearch, Hyperband, BayesianOptimization, and Sklearn. In this tutorial, you use the Hyperband tuner.

In [3]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')    

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Epoch 5/10
1875/1875 [==============================] - ETA: 1s - loss: 2.3582 - accuracy: 0.03 - ETA: 11s - loss: 1.6701 - accuracy: 0.395 - ETA: 12s - loss: 1.3916 - accuracy: 0.507 - ETA: 12s - loss: 1.2430 - accuracy: 0.559 - ETA: 12s - loss: 1.1696 - accuracy: 0.581 - ETA: 12s - loss: 1.1067 - accuracy: 0.599 - ETA: 12s - loss: 1.0526 - accuracy: 0.620 - ETA: 12s - loss: 1.0076 - accuracy: 0.639 - ETA: 12s - loss: 0.9595 - accuracy: 0.658 - ETA: 12s - loss: 0.9355 - accuracy: 0.664 - ETA: 12s - loss: 0.9105 - accuracy: 0.676 - ETA: 12s - loss: 0.8912 - accuracy: 0.684 - ETA: 12s - loss: 0.8791 - accuracy: 0.688 - ETA: 12s - loss: 0.8574 - accuracy: 0.695 - ETA: 12s - loss: 0.8415 - accuracy: 0.700 - ETA: 12s - loss: 0.8304 - accuracy: 0.704 - ETA: 12s - loss: 0.8135 - accuracy: 0.710 - ETA: 12s - loss: 0.8067 - accuracy: 0.714 - ETA: 12s - loss: 0.8012 - accuracy: 0.715 - ETA: 11s - loss: 0.7930 - accuracy: 0.718 - ETA: 11s - loss: 0.7843 - accuracy: 0.720 - ETA: 11s - loss: 0.774

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))